In [ ]:
import numpy as np
import pandas as pd
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
from scipy.spatial import distance
import os

In [2]:
hall=pd.read_csv('df_real_final.csv', low_memory=False)

In [37]:
df=hall.copy()

In [41]:
seat2=seat.loc[seat.seat!='무대', ['x','y', 'seat']]
seat2['label']=0

In [42]:
data=[]
for num in df.label.unique():
    num_data = seat2.values.copy()
    num_data[:, -1] = num
    data.append(num_data)

new_data = np.vstack(data)
new_seat = pd.DataFrame(new_data, columns=['x', 'y', 'seat', 'label'])

In [43]:
new_seat=pd.merge(new_seat, df.loc[:, ['x', 'y', 'label', 'grade', 'seat']], how='left', on=['x', 'y', 'seat', 'label'])

In [49]:
new_seat.fillna('', inplace=True)

In [54]:
%%time
floor=[]
block=[]
seat_line=[]
seat_number=[]
new_seat['floor']=''
new_seat['block']=''
new_seat['seat_line']=0
new_seat['seat_number']=0

for index, row in new_seat.iterrows():
    floor.append(row['seat'].split()[0])
    if row['seat'].split()[1][:3]=='BOX':
        block.append(row['seat'].split()[1][:4])
        seat_line.append(int(row['seat'].split()[1][3]))
    else:
        block.append(row['seat'].split()[1][0])
        seat_line.append(int(row['seat'].split()[1][3:-1]))
    seat_number.append(int(row['seat'].split()[2]))
    
new_seat.floor=floor
new_seat.block=block
new_seat.seat_line=seat_line
new_seat.seat_number=seat_number

CPU times: total: 47.9 s
Wall time: 2min 17s


In [51]:
new_seat['new_grade']=new_seat.grade

In [57]:
def findmin(df, x, y, center, scale):
    min_distance=np.inf
    nearest_grade=''
    if scale==0:
        for index, row in df.iterrows():
            d = distance.euclidean((abs(x-center)/100, y), (abs(row.x-center)/100, row.y))
            if d < min_distance:
                min_distance = d
                nearest_grade = row.grade
    elif scale==1:
        for index, row in df.iterrows():
            d = distance.euclidean((abs(x-center), y/100), (abs(row.x-center), row.y/100))
            if d < min_distance:
                min_distance = d
                nearest_grade = row.grade
    else:
        for index, row in df.iterrows():
            d = distance.euclidean((abs(x-center), y), (abs(row.x-center), row.y))
            if d < min_distance:
                min_distance = d
                nearest_grade = row.grade      
    return nearest_grade

In [58]:
%%time

center=seat[seat.seat=='무대'].x.mean()
seat_none=['']
seat_known=['B_price', 'R_price', 'A_price', 'S_price', 'C_price', 'normal', 'third_level', 'glee',
       'second_level', 'first_level']

for num in sorted(new_seat.label.unique()):
    test=new_seat.loc[(new_seat.label==num), ['grade', 'floor', 'block', 'seat_line', 'seat_number', 'x', 'y']]
    known_seats = test[test.grade.isin(seat_known)]
    if known_seats.empty:
        continue
    # 알려진 좌석과 가장 가까운 좌석 찾아서 채우기
    for index, row in test.iterrows():
        if any(row['grade'] in s for s in seat_none):  # 알려지지 않은 좌석일 경우
            x=row.x
            y=row.y
            if (row['floor']=='1층'):
                known_seat2=known_seats[(known_seats.floor=='1층')]
                if known_seat2.empty==False:
                    if (row.block in ['B', 'C', 'D']):
                        known_seat3=known_seat2[known_seat2.block.isin(['B', 'C', 'D'])]
                        if known_seat3.empty==False:
                            if (any(known_seat3.block=='C'))&(row.block=='C'):
                                known_seat4=known_seat3[known_seat3.block=='C']
                                if known_seat4.empty==False:
                                    df.loc[index, 'new_grade']=findmin(known_seat4, x, y, center, 0)
                                    continue
                            elif (row.block in ['B', 'D'])&(any(known_seat3.block.isin(['B','D']))):
                                known_seat4=known_seat3[known_seat3.block.isin(['B', 'D'])]
                                if known_seat4.empty==False:
                                    df.loc[index, 'new_grade']=findmin(known_seat4, x, y, center, 0)
                                    continue
                            else:
                                df.loc[index, 'new_grade']=findmin(known_seat3, x, y, center, 0)
                                continue
                    else:
                        known_seat3=known_seat2[known_seat2.block.isin(['A', 'E'])]
                        if known_seat3.empty==False:
                            df.loc[index, 'new_grade']=findmin(known_seat3, x, y, center, 1)
                            continue
                    df.loc[index, 'new_grade']=findmin(known_seat2, x, y, center, 2)
                    continue
            elif (row['floor']=='2층'):
                known_seat2=known_seats[(known_seats.floor=='2층')]
                if known_seat2.empty==False:
                    if (row.block in ['B', 'C', 'D']):
                        known_seat3=known_seat2[known_seat2.block.isin(['B', 'C', 'D'])]
                        if known_seat3.empty==False:
                            if (any(known_seat3.block=='C'))&(row.block=='C'):
                                known_seat4=known_seat3[known_seat3.block=='C']
                                if known_seat4.empty==False:
                                    df.loc[index, 'new_grade']=findmin(known_seat4, x, y, center, 0)
                                    continue
                            elif (row.block!='C')&(any(known_seat3.block.isin(['B','D']))):
                                known_seat4=known_seat3[known_seat3.block.isin(['B', 'D'])]
                                if known_seat4.empty==False:
                                    df.loc[index, 'new_grade']=findmin(known_seat4, x, y, center, 0)
                                    continue
                            else:
                                df.loc[index, 'new_grade']=findmin(known_seat3, x, y, center, 0)
                                continue
                    elif (row.block in ['A', 'E']):
                        known_seat3=known_seat2[known_seat2.block.isin(['A', 'E'])]
                        if known_seat3.empty==False:
                            df.loc[index, 'new_grade']=findmin(known_seat3, x, y, center, 1)
                            continue
                    else:
                        known_seat3=known_seat2[known_seat2.block.str.len()>1]
                        if known_seat3.empty==False:
                            df.loc[index, 'new_grade']=findmin(known_seat3, x, y, center, 2)
                            continue
                    df.loc[index, 'new_grade']=findmin(known_seat2, x, y, center, 2)
                    continue
            elif (row['floor']=='3층'):
                known_seat2=known_seats[(known_seats.floor=='3층')]
                if known_seat2.empty==False:
                    if (row.block in ['B', 'C', 'D', 'E', 'F']):
                        known_seat3=known_seat2[known_seat2.block.isin(['B', 'C', 'D', 'E', 'F'])]
                        if known_seat3.empty==False:
                            if (any(known_seat3.block=='D'))&(row.block=='D'):
                                known_seat4=known_seat3[known_seat3.block=='D']
                                if known_seat4.empty==False:
                                    df.loc[index, 'new_grade']=findmin(known_seat4, x, y, center, 0)
                                    continue
                            elif (row.block in ['C','E'])&(any(known_seat3.block.isin(['C','E']))):
                                known_seat4=known_seat3[known_seat3.block.isin(['C', 'E'])]
                                if known_seat4.empty==False:
                                    df.loc[index, 'new_grade']=findmin(known_seat4, x, y, center, 0)
                                    continue
                            elif (row.block in ['B','F'])&(any(known_seat3.block.isin(['B','F']))):
                                known_seat4=known_seat3[known_seat3.block.isin(['B', 'F'])]
                                if known_seat4.empty==False:
                                    df.loc[index, 'new_grade']=findmin(known_seat4, x, y, center, 0)
                                    continue
                            else:
                                df.loc[index, 'new_grade']=findmin(known_seat3, x, y, center, 0)
                                continue
                    elif (row.block in ['A', 'G']):
                        known_seat3=known_seat2[known_seat2.block.isin(['A', 'G'])]
                        if known_seat3.empty==False:
                            df.loc[index, 'new_grade']=findmin(known_seat3, x, y, center, 1)
                            continue
                    else:
                        known_seat3=known_seat2[known_seat2.block.str.len()>1]
                        if known_seat3.empty==False:
                            df.loc[index, 'new_grade']=findmin(known_seat3, x, y, center, 2)
                            continue
                    df.loc[index, 'new_grade']=findmin(known_seat2, x, y, center, 2)
                    continue
            else:
                known_seat2=known_seats[(known_seats.floor=='합창석')]
                if known_seat2.empty==False:
                    if (row.block in ['H', 'F']):
                        known_seat3=known_seat2[known_seat2.block.isin(['H', 'F'])]
                        if known_seat3.empty==False:
                            df.loc[index, 'new_grade']=findmin(known_seat3, x, y, center, 1)
                            continue
                    else:
                        known_seat3=known_seat2[known_seat2.block=='G']
                        if known_seat3.empty==False:
                            df.loc[index, 'new_grade']=findmin(known_seat3, x, y, center, 0)
                            continue
                    df.loc[index, 'new_grade']=findmin(known_seat2, x, y, center, 2)
                    continue
            df.loc[index, 'new_grade']=findmin(known_seats, x, y, center, 2)
            continue

CPU times: total: 1min 38s
Wall time: 3min 58s


In [ ]:
new_seat.to_csv('full_seat.csv', index=False)